<a href="https://colab.research.google.com/github/yvanzhu/e2e-metrics/blob/master/Kopie_von_nmt_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset

In [ ]:
!wget https://www.statmt.org/europarl/v7/nl-en.tgz
!tar -xvzf nl-en.tgz 
!rm nl-en.tgz

# Task1&2 Data exploration and preprocessing

Clean the empty line with moses scripts

In [ ]:
# Get the lines of unprocessed data
#!wc -l europarl-v7.nl-en.en
#!wc -l europarl-v7.nl-en.nl
''' 
Strip the empty lines 
'''
!git clone https://github.com/moses-smt/mosesdecoder.git -q
! ./mosesdecoder/scripts/training/clean-corpus-n.perl europarl-v7.nl-en en nl europarl-v7.nl-en.no_empty 1 10000

Insight of original empty-line stripped dataset 

In [ ]:
import codecs
en = []
nl = []
with codecs.open('europarl-v7.nl-en.no_empty.en', encoding = 'utf-8') as f:
    for line in f:
      en.append(line)
with codecs.open('europarl-v7.nl-en.no_empty.nl', encoding = 'utf-8') as f:
    for line in f:
      nl.append(line) 

In [ ]:
# to dataframe
import pandas as pd 
data_ori = pd.DataFrame
data_ori = pd.DataFrame (en, columns = ['en'])
data_ori['nl'] = pd.DataFrame (nl)
data_ori['en'] = data_ori['en'].str.replace('\n', '')
data_ori['nl'] = data_ori['nl'].str.replace('\n', '')

# Get the length of each sentence
data_ori["en_length"] = data_ori["en"].apply(lambda x: len(x.split()))
data_ori["nl_length"] = data_ori["nl"].apply(lambda x: len(x.split()))

# investigate the distribution of the sentence length
import numpy as np
import matplotlib.pyplot as plt
en_sent = [i for i in data_ori['en_length'] if i != 0]
nl_sent = [i for i in data_ori['nl_length'] if i != 0]
print('Max length in EN is {}'.format(max(en_sent)))
print('Max length in NL is {}'.format(max(nl_sent)))
''' As the empty lines are stripped'''
plt.hist(en_sent, bins = range(min(en_sent), 200, 1), 
              alpha = 0.5, color="red")
plt.hist(nl_sent, bins = range(min(nl_sent), 200, 1),
              alpha = 0.5, color = "blue")
labels = ['EN',"NL"]
plt.legend(labels)
plt.xlabel("length of sentence")
plt.ylabel("proportion")
plt.title("Sentence length distribution of original dataset")


In [ ]:
# clean for some place
!rm europarl-v7.nl-en.no_empty.*

One may know that the long sentence do not improve the translation training. In the following experiments, we decide to compare the performance on length of 15 and 25. We first modify the corpus to file, which only contains sentences with length under 25. Then we will choose approparite ratio to sample the data, in order to make sure it will not out of the capacity of our device.

In [ ]:
''' 
Strip sentences over 25 
Set the min length to 3 to avoid some 
lines only contain the punctuation
'''
! ./mosesdecoder/scripts/training/clean-corpus-n.perl europarl-v7.nl-en en nl europarl-v7.nl-en.clean15 3 15
! ./mosesdecoder/scripts/training/clean-corpus-n.perl europarl-v7.nl-en en nl europarl-v7.nl-en.clean25 3 25

Read data

In [ ]:
en_15 = []
nl_15 = []
en_25 = []
nl_25 = []
with codecs.open('europarl-v7.nl-en.clean15.en', encoding = 'utf-8') as f:
    for line in f:
      en_15.append(line)
with codecs.open('europarl-v7.nl-en.clean15.nl', encoding = 'utf-8') as f:
    for line in f:
      nl_15.append(line) 
with codecs.open('europarl-v7.nl-en.clean25.en', encoding = 'utf-8') as f:
    for line in f:
      en_25.append(line)
with codecs.open('europarl-v7.nl-en.clean25.nl', encoding = 'utf-8') as f:
    for line in f:
      nl_25.append(line) 

In [ ]:
# Sentences length with 25 
data25 = pd.DataFrame
data25 = pd.DataFrame (en_25, columns = ['en'])
data25['nl'] = pd.DataFrame (nl_25)
data25['en'] = data25['en'].str.replace('\n', '')
data25['nl'] = data25['nl'].str.replace('\n', '')
data25

In [ ]:
# Sentences length with 25 
data15 = pd.DataFrame
data15 = pd.DataFrame (en_15, columns = ['en'])
data15['nl'] = pd.DataFrame (nl_15)
data15['en'] = data15['en'].str.replace('\n', '')
data15['nl'] = data15['nl'].str.replace('\n', '')
data15

In [ ]:
# Get 100000 pairs as sampled data
sampled_data_25 = data25.sample(n = 100000, ignore_index = True, random_state = 1)
sampled_data_15 = data15.sample(n = 100000, ignore_index = True, random_state = 1)

In [ ]:
# Convert the data to pair format
X = sampled_data_25['en']
y = sampled_data_25['nl']

with open ('sampled_data_25.ennl', 'w') as f:
  for lineen, linenl in zip(X, y):
    f.write(lineen+ '\t'+ linenl+ '\n')
  f.close()

X = sampled_data_15['en']
y = sampled_data_15['nl']

with open ('sampled_data_15.ennl', 'w') as f:
  for lineen, linenl in zip(X, y):
    f.write(lineen+ '\t'+ linenl+ '\n')
  f.close()

# Task 3

The following experiments are not strictly follw the task description. Every detail is explained by text

In [ ]:
data_25_path = "./sampled_data_25.ennl"
data_15_path = "./sampled_data_15.ennl"

In [ ]:
!pip install sacrebleu -q
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits

__Clean and Preprocess the text__
1. Convert to lower case
2. Remove spaces
3. Remove punctuations
3. Add start_ and end_ tags to each sentence


In [ ]:
import string
def preprocess_sentence(sentence):
    #sentence = unicode_to_ascii(sentence.lower().strip())
    #num_digits= str.maketrans('','', digits)
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    #sentence= sentence.translate(num_digits)
    sentence= sentence.strip()
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence= ' '.join(sentence.split())
    #for punctuation in string.punctuation:
      #sentence = sentence.replace(punctuation, '')
    sentence=  'start_ ' + sentence + ' _end'
    
    return sentence

In [ ]:
en_sentence = u"I will go home at 7 o'clock."
nl_sentence = u"Ik ga om 7 uur naar huis."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(nl_sentence).encode('utf-8'))

In [ ]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #print(lines)
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
  print(path)
  return zip(*word_pairs)

In [ ]:
sample_size=80000
source, target = create_dataset(data_25_path, sample_size)
source_15, target_15 = create_dataset(data_15_path, sample_size)
print(source[-1])
print(target[-1])

Create the source and target tokens and post pad them

In [ ]:
MAX_LEN = 27 # plus start_ and _end
source_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
source_sentence_tokenizer.fit_on_texts(source)
source_tensor = source_sentence_tokenizer.texts_to_sequences(source)
source_tensor= tf.keras.preprocessing.sequence.pad_sequences(source_tensor,maxlen = MAX_LEN,padding='post' )

In [ ]:
target_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
target_sentence_tokenizer.fit_on_texts(target)
target_tensor = target_sentence_tokenizer.texts_to_sequences(target)
target_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_tensor,maxlen = MAX_LEN,padding='post' )


Get the max_length of tensor. (MAX_LENGTH = Length +`<START>` + `<END>`)

In [ ]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [ ]:
max_target_length= max(len(t) for t in  target_tensor)
print(max_target_length)
max_source_length= max(len(t) for t in  source_tensor)
print(max_source_length)

### Word embedding on source side

In [ ]:
vocab_inp_size = len(source_sentence_tokenizer.word_index)+1
vocab_tar_size = len(target_sentence_tokenizer.word_index)+1

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.zip

embeddings_file_path = 'glove.6B.300d.txt'

In [ ]:
import numpy as np
from numpy import array, asarray, zeros

embeddings_dict = dict()
glove_file = open(embeddings_file_path)

for line in glove_file:
    records = line.split() # Turn into array with word on first position and embeddings as rest of line.
    word = records[0]
    vector_dim = asarray(records[1:], dtype='float32') # Take rest of embeddings out.
    embeddings_dict[word] = vector_dim # Add to embeddings_dict as word:embeddings.

In [ ]:
# Make sure your embedding size < word embeddings for each line.
EMBEDDING_SIZE = 300
word_index_dict = source_sentence_tokenizer.word_index
word_count = len(word_index_dict) + 1

# num_words is decided either by the minimum word count or the amount of words
# in the input sentences, whatever is smaller.
#word_count = min(VOCABULARY_SIZE, word_count)

# Creates a matrix of zeroes based on the English word count and the 
# size of the embeddings.
embeddings_matrix = zeros((word_count, EMBEDDING_SIZE)) 
for word, index in word_index_dict.items():
    # Attempts to get word embeddings for specific word.
    embeddings_vector = embeddings_dict.get(word)
    if embeddings_vector is not None:
        embeddings_matrix[index] = embeddings_vector
word_count

### Word Embedding of Target side

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nl.300.vec.gz
!gunzip -d cc.nl.300.vec.gz
nl_embeddings_file_path ='cc.nl.300.vec'

In [ ]:
nl_embeddings_dict = dict()
glove_nl_file = open(nl_embeddings_file_path)

for line in glove_nl_file:
    records = line.split() # Turn into array with word on first position and embeddings as rest of line.
    word = records[0]
    vector_dim = asarray(records[1:], dtype='float32') # Take rest of embeddings out.
    nl_embeddings_dict[word] = vector_dim # Add to embeddings_dict as word:embeddings.

In [ ]:
# Make sure your embedding size < word embeddings for each line.
EMBEDDING_SIZE_nl = 300
word_index_dict_nl = target_sentence_tokenizer.word_index
word_count_nl = len(word_index_dict_nl) + 1

# num_words is decided either by the minimum word count or the amount of words
# in the input sentences, whatever is smaller.
#word_count = min(VOCABULARY_SIZE, word_count)

# Creates a matrix of zeroes based on the English word count and the 
# size of the embeddings.
embeddings_matrix_nl = zeros((word_count_nl, EMBEDDING_SIZE_nl)) 
for word, index in word_index_dict_nl.items():
    # Attempts to get word embeddings for specific word.
    embeddings_vector = nl_embeddings_dict.get(word)
    if embeddings_vector is not None:
        embeddings_matrix_nl[index] = embeddings_vector
word_count_nl

##  Creating Train, Validation and Test dataset

In [ ]:
# Token-level splitted reference same as tensor split
source_train, source_val, target_train, target_val = train_test_split(source, target, test_size=0.4,random_state = 1 )
source_val,source_test,target_val,target_test = train_test_split(source_val, target_val, test_size=0.5,random_state = 1 )

# Creating training, validation and test sets using an 60-20-20 split
source_train_tensor, source_val_tensor, target_train_tensor, target_val_tensor= train_test_split(source_tensor, target_tensor,test_size=0.4,random_state = 1)
source_val_tensor, source_test_tensor, target_val_tensor, target_test_tensor = train_test_split(source_val_tensor, target_val_tensor, test_size=0.5,random_state = 1)
print(len(source_train_tensor), len(source_val_tensor), len(source_test_tensor))

Examples of word mapping

In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
 

In [ ]:
print ("Input Language; index to word mapping")
convert(source_sentence_tokenizer, source_train_tensor[0])
print ()
print ("Target Language; index to word mapping")
convert( target_sentence_tokenizer, target_train_tensor[0])

### Create a tf.data dataset

In [ ]:
MAX_LEN = 27
BUFFER_SIZE = len(source_train_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(source_train_tensor)//BATCH_SIZE
steps_per_epoch_val = len(source_val_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(source_sentence_tokenizer.word_index)+1
vocab_tar_size = len(target_sentence_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((source_train_tensor, target_train_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((source_val_tensor, target_val_tensor)).shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)
 
print('The size of EN vocab is {}'.format(vocab_inp_size))
print('The size of NL vocab is {}'.format(vocab_tar_size))
print('The steps per training epoch is {}'.format(steps_per_epoch))
print('The steps per validation epoch is {}'.format(steps_per_epoch_val))

In [ ]:
# Example batch for configure the model
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

## The encoder and decoder model




### Encoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, max_len, word_emb = 0,reverse = 0): #word_emb for imply the use of word_embedding
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    if reverse ==1 and word_emb ==1:
      self.embedding = tf.keras.layers.Embedding(vocab_tar_size,
                                               EMBEDDING_SIZE_nl,
                                               weights=[embeddings_matrix_nl],
                                               input_length=max_len)
    if word_emb ==1 and reverse ==0:
      self.embedding = tf.keras.layers.Embedding(vocab_inp_size,
                                               EMBEDDING_SIZE,
                                               weights=[embeddings_matrix],
                                               input_length=max_len)
    if word_emb ==0 and reverse ==0:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
enc = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, MAX_LEN, word_emb = 0, reverse = 0)

# sample input
sample_hidden = enc.initialize_hidden_state()
sample_output, sample_hidden = enc(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [ ]:
enc.summary()

### Decoder 

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, max_len, word_emb = 0, reverse = 0):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    if reverse ==1 and word_emb ==1:
      self.embedding = tf.keras.layers.Embedding(vocab_inp_size,
                                               EMBEDDING_SIZE,
                                               weights = [embeddings_matrix],
                                               input_length = max_len)
    if reverse ==0 and word_emb ==1:
      self.embedding = tf.keras.layers.Embedding(vocab_tar_size,
                                               EMBEDDING_SIZE_nl,
                                               weights = [embeddings_matrix_nl],
                                               input_length = max_len)
    if reverse ==0 and word_emb ==0:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden, enc_output):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state = hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [ ]:
dec = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, MAX_LEN, word_emb = 0,reverse = 0)

sample_decoder_output, _= dec(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
dec.summary()

## Define the optimizer and the loss function

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2) 

In [ ]:
learning_rate = CustomSchedule(d_model = 1024)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)
'''def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=1))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)'''

 

## Checkpoints (Object-based saving)

In [ ]:
#checkpoint_dir = 'training_checkpoints'
#checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
#checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                 encoder=encoder,
#                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [ ]:
def train_step(inp, targ, enc_hidden, encoder, decoder, attention = 0, reverse = 0):
  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden
    if reverse ==1:
      dec_input = tf.expand_dims([source_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)
    if reverse ==0:
      dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      if attention:
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      else:
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
      
      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

 
  return batch_loss 

In [ ]:
@tf.function
def caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder, attention = 0, reverse = 0 ):
  loss = 0
  enc_output, enc_hidden = encoder(inp, enc_hidden)
  dec_hidden = enc_hidden
  if reverse ==1:
    dec_input = tf.expand_dims([source_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)
  if reverse ==0:
    dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  for t in range(1, targ.shape[1]):
    if attention:
      predictions, dec_hidden, _  = decoder(dec_input, dec_hidden, enc_output)
    else:
      predictions, dec_hidden  = decoder(dec_input, dec_hidden, enc_output)
    loss += loss_function(targ[:, t], predictions)
    dec_input = tf.expand_dims(targ[:, t], 1)

  loss = loss / int(targ.shape[1])
 
  return loss

In [ ]:
#Define training function
def train(epochs, encoder,decoder, attention=0, reverse = 0):
  encoder = encoder 
  decoder = decoder
  training_loss  =[]
  validation_loss =[]
  for epoch in range(epochs):
    start = time.time()
    
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    if reverse ==1:
      for (batch, (targ, inp)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss   = train_step(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
        total_loss  += batch_loss 
        if batch % 500 == 0:
          print('Epoch {} Batch {} loss {:.4f} '.format(epoch + 1,batch, batch_loss.numpy() ))
    if reverse ==0:
      for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss   = train_step(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
        total_loss  += batch_loss 
        if batch % 500 == 0:
          print('Epoch {} Batch {} loss {:.4f} '.format(epoch + 1,batch, batch_loss.numpy() ))
        

  
    enc_hidden = encoder.initialize_hidden_state()
    total_val_loss  = 0
    if reverse ==1:
      for (batch, (targ, inp)) in enumerate(validation_dataset.take(steps_per_epoch_val)):
        val_loss  = caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
        total_val_loss  += val_loss 
    if reverse ==0:
      for (batch, (inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch_val)):
        val_loss  = caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
        total_val_loss  += val_loss 

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1])) 

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder, decoder, training_loss , validation_loss

In [ ]:
print('TRAINING WITHOUT ATTENTION LAYER') 
ENCODER = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 0, reverse = 0)
DECODER = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 0, reverse = 0)
EPOCHS = 10
encoder_no_att, decoder_no_att, train_loss_no_att, val_loss_no_att = train(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 0, reverse = 0)


In [ ]:
print('TRAINING WITH GLOVE WITHOUT ATTENTION LAYER ') 
ENCODER = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 1, reverse = 0)
DECODER = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 1, reverse = 0)
EPOCHS = 10
encoder_emb, decoder_emb, train_loss_emb, val_loss_emb = train(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 0,reverse = 0)


In [ ]:
epochs = 10
ax = plt.subplot(111) 
 
t = np.arange(1, epochs+1,1)
plt.plot(t, train_loss_no_att ,t, val_loss_no_att , lw=2)
plt.plot(t, train_loss_emb,t, val_loss_emb , lw=2, )
 

plt.gcf().set_size_inches(10, 10)
ax.set_title("Train & Validation loss in 10 epochs")
ax.legend(['Train_loss', 'Val_loss','Train_loss_emb','val_loss_emb'])

### Tranlation

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token* or when the max traget legth is reached
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [ ]:
def evaluate(sentence, encoder, decoder, max_len, attention = 0, reverse = 0):
  tf.debugging.set_log_device_placement(True)
  with tf.device('/device:GPU:0'):
    attention_plot = np.zeros((max_len, max_len))

    sentence = preprocess_sentence(sentence)
  #print(sentence)
  #print(source_sentence_tokenizer.word_index)
    if reverse ==1:
      inputs = [target_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]
    if reverse ==0:
      inputs = [source_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]

    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_len,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    if reverse ==1:
      dec_input = tf.expand_dims([source_sentence_tokenizer.word_index['start_']], 0)
    if reverse ==0:
      dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']], 0)

    for t in range(max_target_length):
      if attention:
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
      else:
        predictions, dec_hidden = decoder(dec_input,
                                          dec_hidden,
                                          enc_out)

      predicted_id = tf.argmax(predictions[0]).numpy()
      if reverse ==1:
        result += source_sentence_tokenizer.index_word[predicted_id] + ' '
      if reverse ==0:
        result += target_sentence_tokenizer.index_word[predicted_id] + ' '
      if reverse ==1:
        if source_sentence_tokenizer.index_word[predicted_id] == '_end':
          if attention: return result, sentence, attention_plot
          else: return result, sentence
      if reverse ==0:
        if target_sentence_tokenizer.index_word[predicted_id] == '_end':
          if attention: return result, sentence, attention_plot
          else: return result, sentence      

    # the predicted ID is fed back into the model
      dec_input = tf.expand_dims([predicted_id], 0)

    if attention:
      return result, sentence, attention_plot
    else:
      return result, sentence

In [ ]:
# Translation with only enc-dec
result, sentence  = evaluate(u'The Ombudsman must increase its efforts to provide the public with reliable information.', encoder_no_att, decoder_no_att, max_len = MAX_LEN, attention = 0, reverse = 0)
result

In [ ]:
# Translation with enc-dec-emb
result, _  = evaluate(u'The Ombudsman must increase its efforts to provide the public with reliable information.', encoder_emb, decoder_emb, max_len = MAX_LEN, attention = 0, reverse = 0)
result

### Results with metrics

In [ ]:
# Get the translated text of validation set and test set

tf.debugging.set_log_device_placement(True)
with tf.device('/device:GPU:0'):
  translated_val = []
  translated_val_emb = []
  for j in source_val[0:1000]:
    result, _  = evaluate(j, encoder_no_att, decoder_no_att, max_len = MAX_LEN, attention = 0, reverse = 0)
    result_emb, _  = evaluate(j, encoder_emb, decoder_emb, max_len = MAX_LEN, attention = 0, reverse = 0)
    translated_val.append(result.replace(' _end',''))
    translated_val_emb.append(result_emb.replace(' _end',''))
  with open('translation_val.txt','w') as f:
    for line in translated_val:
      line = line.replace(' _end','')
      f.write(line+'\n')
  with open('translation_val_emb.txt','w') as f:
    for line in translated_val_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')
  
  translated_test = []
  translated_test_emb = []
  for i in source_test[0:1000]:
    result, _  = evaluate(i, encoder_no_att, decoder_no_att, max_len = MAX_LEN, attention = 0, reverse = 0)
    result_emb, _  = evaluate(i, encoder_emb, decoder_emb, max_len = MAX_LEN, attention = 0, reverse = 0)
    translated_test.append(result.replace(' _end',''))
    translated_test_emb.append(result_emb.replace(' _end',''))
  with open('translation_test.txt','w') as f:
    for line in translated_test:
      line = line.replace(' _end','')
      f.write(line+'\n')
  with open('translation_test_emb.txt','w') as f:
    for line in translated_test_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')


In [ ]:
with open('reference_val.txt','w') as f:
  for line in target_val[0:1000]:
    line = line.replace(' _end','')
    line = line.replace('start_ ','')
    f.write(line+'\n')

with open('reference_test.txt','w') as f:
  for line in target_test[0:1000]:
    line = line.replace(' _end','')
    line = line.replace('start_ ','')
    f.write(line+'\n')

In [ ]:
'''
score :
[ bleu
chrf2
ter ]
'''
print('Score on Validation set only enc-dec of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_val.txt' -i 'translation_val.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Test set only enc-dec of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_test.txt' -i 'translation_test.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Validation set only enc-dec+emb of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_val.txt' -i 'translation_val_emb.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Test set only enc-dec+emb of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_test.txt' -i 'translation_test_emb.txt' -m bleu ter chrf -w 4 -b --force

### Task 3.2 Investigation on Length
So far, the translation based on sentence length of 25 is done. Now we turn to the length of 15 to compare the performance. Just trunctuate the sentence into 15.

In [ ]:
MAX_LEN_15 = 17 # plus start_ and _end
#source_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
#source_sentence_tokenizer.fit_on_texts(source)
source_tensor_15 = source_sentence_tokenizer.texts_to_sequences(source)
source_tensor_15= tf.keras.preprocessing.sequence.pad_sequences(source_tensor_15, maxlen = MAX_LEN_15, padding='post' )

#target_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
#target_sentence_tokenizer.fit_on_texts(target)
target_tensor_15 = target_sentence_tokenizer.texts_to_sequences(target)
target_tensor_15 = tf.keras.preprocessing.sequence.pad_sequences(target_tensor_15, maxlen = MAX_LEN_15, padding='post' )


# Token-level splitted reference same as tensor split
source_train_15, source_val_15, target_train_15, target_val_15 = train_test_split(source, target, test_size=0.4,random_state = 1 )
source_val_15,source_test_15,target_val_15,target_test_15 = train_test_split(source_val_15, target_val_15, test_size=0.5,random_state = 1 )

# Creating training and validation sets using an 80-20 split
source_train_tensor_15, source_val_tensor_15, target_train_tensor_15, target_val_tensor_15= train_test_split(source_tensor_15, target_tensor_15,test_size=0.4,random_state = 1)
source_val_tensor_15, source_test_tensor_15, target_val_tensor_15, target_test_tensor_15 = train_test_split(source_val_tensor_15, target_val_tensor_15, test_size=0.5,random_state = 1)

dataset_15 = tf.data.Dataset.from_tensor_slices((source_train_tensor_15, target_train_tensor_15)).shuffle(BUFFER_SIZE)
dataset_15 = dataset_15.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset_15 = tf.data.Dataset.from_tensor_slices((source_val_tensor_15, target_val_tensor_15)).shuffle(BUFFER_SIZE)
validation_dataset_15 = validation_dataset_15.batch(BATCH_SIZE, drop_remainder=True)
 


In [ ]:
#Define training function
def train_15(epochs, encoder, decoder, attention=0, reverse = 0):
  encoder = encoder 
  decoder = decoder
  training_loss  =[]
  validation_loss =[]
  for epoch in range(epochs):
    start = time.time()
    
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset_15.take(steps_per_epoch)):
      batch_loss   = train_step(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
      total_loss  += batch_loss 
      if batch % 500 == 0:
        print('Epoch {} Batch {} loss {:.4f} '.format(epoch + 1,batch, batch_loss.numpy() ))
        

  
    enc_hidden = encoder.initialize_hidden_state()
    total_val_loss  = 0
    for (batch, (inp, targ)) in enumerate(validation_dataset_15.take(steps_per_epoch_val)):
      val_loss  = caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder, attention, reverse)
      total_val_loss  += val_loss 

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1])) 

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder, decoder, training_loss , validation_loss

In [ ]:
print('TRAINING WITH GLOVE WITHOUT ATTENTION LAYER L = 15 ') 
ENCODER = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, MAX_LEN_15, word_emb = 1,reverse = 0)
DECODER = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, MAX_LEN_15, word_emb = 1,reverse = 0)
EPOCHS = 10
encoder_emb_15, decoder_emb_15, train_loss_emb_15, val_loss_emb_15 = train_15(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 0)


In [ ]:
epochs = 10
ax = plt.subplot(111) 
 
t = np.arange(1, epochs+1,1)
plt.plot(t, train_loss_emb_15,t, val_loss_emb_15 , lw=2)
plt.plot(t, train_loss_emb,t, val_loss_emb , lw=2, )
 

plt.gcf().set_size_inches(10, 10)
ax.set_title("Train & Validation loss in 10 epochs")
ax.legend(['Train_loss_L_15', 'Val_loss_L_15','Train_loss_L_25','val_loss_L_25'])

In [ ]:
# Get the translated text of validation set and test set

tf.debugging.set_log_device_placement(True)
with tf.device('/device:GPU:0'):
  translated_val_emb = []
  sentence_val = []
  for j in source_val_15[0:1000]:
    result_emb, sentence  = evaluate(j, encoder_emb_15, decoder_emb_15, max_len = MAX_LEN_15, attention = 0, reverse = 0)
    translated_val_emb.append(result_emb.replace(' _end',''))
    sentence = sentence.replace(' _end','')
    sentence_val.append(sentence.replace('start_ ',''))
  with open('translation_val_emb_15.txt','w') as f:
    for line in translated_val_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')
  with open('reference_val_15.txt','w') as f:
    for line in sentence_val:
      f.write(line+'\n')
  
  translated_test_emb = []
  sentence_test = []
  for i in source_test_15[0:1000]:
    result_emb, sentence  = evaluate(i, encoder_emb_15, decoder_emb_15, max_len = MAX_LEN_15, attention = 0, reverse = 0)
    translated_test_emb.append(result_emb.replace(' _end',''))
    sentence = sentence.replace(' _end','')
    sentence_test.append(sentence.replace('start_ ',''))
  with open('translation_test_emb_15.txt','w') as f:
    for line in translated_test_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')
  with open('reference_test_15.txt','w') as f:
    for line in sentence_test:
      f.write(line+'\n')

In [ ]:
print('Score on Validation set only enc-dec+emb_L=15 of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_val_15.txt' -i 'translation_val_emb_15.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Test set only enc-dec+emb_L=15 of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_test_15.txt' -i 'translation_test_emb_15.txt' -m bleu ter chrf -w 4 -b --force

###Task 3.3 Reversed Translation

In [ ]:
print('REVERSED TRAINING WITH GLOVE WITHOUT ATTENTION LAYER ') 
ENCODER = Encoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, MAX_LEN, word_emb = 1,reverse = 1)
DECODER = Decoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, MAX_LEN, word_emb = 1,reverse = 1)
EPOCHS = 10
encoder_reverse, decoder_reverse, train_loss_reverse, val_loss_reverse = train(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 0, reverse = 1)


In [ ]:
epochs = 10
ax = plt.subplot(111) 
 
t = np.arange(1, epochs+1,1)
plt.plot(t, train_loss_reverse,t, val_loss_reverse , lw=2)
plt.plot(t, train_loss_emb,t, val_loss_emb , lw=2, )
 

plt.gcf().set_size_inches(10, 10)
ax.set_title("Train & Validation loss in 10 epochs")
ax.legend(['Train_loss_nl-en', 'Val_loss_nl-en','Train_loss_en-nl','val_loss_en-nl'])

In [ ]:
result, sentence  = evaluate(u'Het is dan ook volkomen juist om door te gaan.',encoder_reverse, decoder_reverse, max_len=MAX_LEN, attention = 0, reverse =1)
result


In [ ]:
# Get the translated text of validation set and test set

tf.debugging.set_log_device_placement(True)
with tf.device('/device:GPU:0'):
  translated_val_emb = []
  for j in target_val[0:2000]:
    result_emb, _  = evaluate(j, encoder_reverse, decoder_reverse, max_len = MAX_LEN, attention = 0, reverse = 1)
    translated_val_emb.append(result_emb.replace(' _end',''))
  with open('translation_val_reverse.txt','w') as f:
    for line in translated_val_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')
  
  translated_test_emb = []
  for i in target_test[0:2000]:
    result_emb, _  = evaluate(i,encoder_reverse, decoder_reverse, max_len = MAX_LEN, attention = 0, reverse = 1)
    translated_test_emb.append(result_emb.replace(' _end',''))

  with open('translation_test_reverse.txt','w') as f:
    for line in translated_test_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')

In [ ]:
with open('reference_val_reverse.txt','w') as f:
  for line in source_val[0:2000]:
    line = line.replace(' _end','')
    line = line.replace('start_ ','')
    f.write(line+'\n')

with open('reference_test_reverse.txt','w') as f:
  for line in source_test[0:2000]:
    line = line.replace(' _end','')
    line = line.replace('start_ ','')
    f.write(line+'\n')

In [ ]:
print('Reversed translation from NL to EN')
print('Score on Validation set reversed enc-dec+emb_L=25 of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_val_reverse.txt' -i 'translation_val_reverse.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Test set reversed enc-dec+emb_L=25 of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_test_reverse.txt' -i 'translation_test_reverse.txt' -m bleu ter chrf -w 4 -b --force

### Task 3.4 Charater-level tokenizer

In [ ]:
MAX_LEN = 27 # plus start_ and _end
source_char_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='',char_level=True)
source_char_tokenizer.fit_on_texts(source)
source_char_tensor = source_char_tokenizer.texts_to_sequences(source)
source_char_tensor= tf.keras.preprocessing.sequence.pad_sequences(source_char_tensor,padding='post' )

In [ ]:
MAX_LEN = 27 # plus start_ and _end
target_char_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='',char_level=True)
target_char_tokenizer.fit_on_texts(target)
target_char_tensor = target_char_tokenizer.texts_to_sequences(target)
target_char_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_char_tensor,padding='post' )

In [ ]:
max_target_length_char = max(len(t) for t in  target_char_tensor)
print(max_target_length_char)
max_source_length_char = max(len(t) for t in  source_char_tensor)
print(max_source_length_char)

In [ ]:
vocab_inp_size_char = len(source_char_tokenizer.word_index)+1
vocab_tar_size_char = len(target_char_tokenizer.word_index)+1
print(vocab_inp_size_char)
print(vocab_tar_size_char)

In [ ]:
print('TRAINING WITH GLOVE WITHOUT ATTENTION LAYER ') 
ENCODER = Encoder(vocab_inp_size_char, embedding_dim, units, BATCH_SIZE, max_len = max_source_length_char, word_emb = 0, reverse = 0)
DECODER = Decoder(vocab_tar_size_char, embedding_dim, units, BATCH_SIZE, max_len = max_target_length_char, word_emb = 0, reverse = 0)
EPOCHS = 3
encoder_char, decoder_char, train_loss_char, val_loss_char = train(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 0,reverse = 0)


## Restore the latest checkpoint and test

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Task 4 Attention layer

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [ ]:
class AttentionDecoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, max_len, word_emb = 0, reverse = 0):
    super(AttentionDecoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    if reverse:
      self.embedding = tf.keras.layers.Embedding(vocab_inp_size,
                                               EMBEDDING_SIZE,
                                               weights = [embeddings_matrix],
                                               input_length = max_len)
    if word_emb:
      self.embedding = tf.keras.layers.Embedding(vocab_tar_size,
                                               EMBEDDING_SIZE_nl,
                                               weights = [embeddings_matrix_nl],
                                               input_length = max_len)
    else:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x,initial_state = hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
att_dec = AttentionDecoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, MAX_LEN, word_emb = 0, reverse = 0)

sample_decoder_output, _, _ = att_dec(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
att_dec.summary()

In [ ]:
print('TRAINING WITH GLOVE WITH ATTENTION LAYER ') 
ENCODER = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 1, reverse = 0)
DECODER = AttentionDecoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, max_len = MAX_LEN, word_emb = 1,reverse = 0)
EPOCHS = 5
encoder_att, decoder_att, train_loss_att, val_loss_att = train(epochs = EPOCHS, encoder = ENCODER, decoder = DECODER, attention = 1)


In [ ]:
epochs = 10
ax = plt.subplot(111) 
 
t = np.arange(1, epochs+1,1)
plt.plot(t, train_loss_att,t, val_loss_att, lw=2)
plt.plot(t, train_loss_emb,t, val_loss_emb , lw=2, )
 
plt.gcf().set_size_inches(10, 10)
ax.set_title("Train & Validation loss in 10 epochs")
ax.legend(['Train_loss_att', 'Val_loss_att','Train_loss_no_att','val_loss_no_att'])

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  cax = ax.matshow(attention, cmap='bone')
  fig.colorbar(cax)

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentence,encoder,decoder,attention = 1):
  result, sentence, attention_plot = evaluate(sentence,encoder,decoder,attention = 1)
  
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
translate(u'The Ombudsman must increase its efforts to provide the public with reliable information.',encoder_att, decoder_att,attention = 1)

In [ ]:
tf.debugging.set_log_device_placement(True)
with tf.device('/device:GPU:0'):
  translated_val_emb = []
  for j in source_val[0:1000]:
    result_emb, _ ,_  = evaluate(j, encoder_att, decoder_att, max_len = MAX_LEN, attention = 1, reverse = 0)
    translated_val_emb.append(result_emb.replace(' _end',''))
  with open('translation_val_att.txt','w') as f:
    for line in translated_val_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')
  
  translated_test_emb = []
  for i in source_test[0:1000]:
    result_emb, _ ,_ = evaluate(i, encoder_att, decoder_att, max_len = MAX_LEN, attention = 1, reverse = 0)
    translated_test_emb.append(result_emb.replace(' _end',''))

  with open('translation_test_att.txt','w') as f:
    for line in translated_test_emb:
      line = line.replace(' _end','')
      f.write(line+'\n')


In [ ]:
print('Score on Validation set only enc-dec+emb+att of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_val.txt' -i 'translation_val_att.txt' -m bleu ter chrf -w 4 -b --force
print('Score on Test set only enc-dec+emb+att of Bleu, ChrF2 and TER are ')
!sacrebleu 'reference_test.txt' -i 'translation_test_att.txt' -m bleu ter chrf -w 4 -b --force